# Guía del proyecto Estadia / Aprendia Edge Backend

Este notebook documenta el **flujo de trabajo**, **inputs/outputs**, **cómo probar** el API y el **funcionamiento de las métricas** del backend de evaluación automática de caligrafía.

## 1. Flujo de trabajo

El sistema compara el **trazo dibujado por el alumno** (imagen) con una **plantilla de referencia** (esqueleto del carácter) y devuelve una puntuación y métricas.

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│  Imagen PNG/JPG │ ──► │  Preprocesamiento│ ──► │  Esqueleto      │
│  (trazo alumno) │     │  128×128, blur   │     │  del alumno     │
└─────────────────┘     │  + esqueletización│     └────────┬────────┘
                        └──────────────────┘              │
┌─────────────────┐     ┌──────────────────┐               │
│  Plantilla .npy │ ──► │  Cache o carga   │ ──────────────┤
│  (128×128)      │     │  get_template()  │               │
└─────────────────┘     └──────────────────┘               ▼
┌─────────────────────────────────────────────────────────────────────┐
│  MÉTRICAS (128×128)                                                  │
│  SSIM · Procrustes · Hausdorff · Topología · DTW (banda) · Coseno   │
│  · Calidad (aspect ratio, Hu)                                        │
└─────────────────────────────────────────────────────────────────────┘
                                                           │
                                                           ▼
┌─────────────────────────────────────────────────────────────────────┐
│  Score final ponderado + imagen comparación Base64 (128×128)         │
└─────────────────────────────────────────────────────────────────────┘
```

### Pasos en detalle

1. **Entrada**: el cliente envía una imagen (trazo del alumno) y el carácter objetivo (`target_char`).
2. **Carga de plantilla**: desde **cache en memoria** (`TEMPLATE_CACHE`) si el carácter ya se cargó; si no, se lee el `.npy` desde `app/templates/`, se redimensiona a **128×128** si era 256×256 y se guarda en cache.
3. **Preprocesamiento** (`preprocess_robust` en `processor.py`), **alineado con el generador de plantillas**:
   - Decodificación, binarización adaptativa (o detección de imagen ya binaria/esqueleto).
   - Recorte al bounding box, centrado en canvas cuadrado, redimensionado a **128×128** (config `TARGET_SHAPE`).
   - **GaussianBlur(9,9)** y **threshold 110** (mismo que plantillas).
   - **Esqueletización** (método Lee) y **poda de espolones** con `MIN_BRANCH_LENGTH=10`.
4. **Cálculo de métricas**: SSIM, Procrustes, Hausdorff, topología, trayectoria (DTW con banda), similitud de coseno por segmentos, calidad.
5. **Score final**: combinación ponderada en `scorer.py` (Procrustes > Hausdorff, SSIM, topo, traj, coseno).
6. **Salida**: JSON con `score_final`, métricas e imagen Base64 de comparación (128×128).

## 2. Inputs y outputs

### Endpoint: `POST /evaluate`

| Parámetro     | Tipo   | Obligatorio | Descripción |
|---------------|--------|-------------|-------------|
| `file`        | File   | Sí          | Imagen del trazo (PNG, JPG). Formato: multipart/form-data. |
| `target_char` | string | Sí (Form)   | Carácter a evaluar: letra (A–Z, a–z, Ñ, ñ) o dígito (0–9). |

### Respuesta exitosa (200)

```json
{
  "char": "a",
  "score_final": 78.45,
  "metrics": {
    "geometric": {
      "ssim": 0.82,
      "ssim_score": 91.0,
      "procrustes_disparity": 0.012,
      "procrustes_score": 99.4,
      "hausdorff": 3.5,
      "score": 94.75
    },
    "topology": { "match": true, "student": {...}, "pattern": {...} },
    "quality": { "aspect_ratio": 0.85, "shape_fingerprint": [...] },
    "trajectory_error": 2.1,
    "segment_cosine": { "cosine": 0.92, "score": 96.0 }
  },
  "image_b64": "iVBORw0KGgoAAAANSUhEUgAA..."
}
```

### Errores

| Código | Condición | Ejemplo |
|--------|-----------|---------|
| 404 | No existe plantilla para el carácter | `"No existe plantilla para 'X'"` |
| 200 | Imagen sin trazo detectable | `{"error": "Sin trazo detectado"}` en el cuerpo |

## 3. Cómo probarlo

### Requisitos

- Python 3.x
- Dependencias: `pip install -r requirements.txt`

### Opción A: Levantar el servidor y usar Swagger

1. En una terminal, desde la raíz del proyecto:
   ```bash
   uvicorn app.main:app --reload
   ```
2. Abre **http://127.0.0.1:8000/docs** (Swagger UI).
3. Expande `POST /evaluate`, pulsa **Try it out**, sube un archivo en `file` y escribe el carácter en `target_char`.
4. Ejecuta y revisa el JSON; puedes decodificar `image_b64` para ver la comparación visual.

### Opción B: Probar con curl

```bash
curl -X POST "http://127.0.0.1:8000/evaluate" \
  -F "file=@ruta/a/tu_imagen_trazo.png" \
  -F "target_char=a"
```

### Opción C: Desde Python (requests)

En la siguiente celda se muestra un ejemplo con `requests` usando una imagen del proyecto.

In [ ]:
# Ejemplo: llamar al API desde Python (el servidor debe estar corriendo en http://127.0.0.1:8000)
import requests

BASE_URL = "http://127.0.0.1:8000"
# Usar una plantilla PNG como "trazo de prueba" (o cualquier imagen con un trazo)
with open("app/templates/a_lower.png", "rb") as f:
    files = {"file": ("a_lower.png", f, "image/png")}
    data = {"target_char": "a"}
    r = requests.post(f"{BASE_URL}/evaluate", files=files, data=data)

if r.status_code == 200 and "error" not in r.json():
    j = r.json()
    print("Carácter:", j["char"])
    print("Score final:", j["score_final"])
    print("Geométricas (SSIM, Procrustes, Hausdorff):", j["metrics"]["geometric"])
    print("Segment cosine:", j["metrics"]["segment_cosine"])
    print("Topología (match):", j["metrics"]["topology"]["match"])
else:
    print("Respuesta:", r.status_code, r.json())

### Generar plantillas (opcional)

Pipeline **alineado con el preprocesamiento del alumno** (mismo blur, umbral y poda):

```bash
python -m app.scripts.generate_templates
```

Genera/sobrescribe `.npy` y `.png` en **128×128** en `app/templates/` para A–Z, Ñ, a–z, ñ, 0–9. Usa `GaussianBlur(9,9)`, `threshold(110)` y `prune_skeleton(min_branch_length=10)` como en `processor.py`.

## 4. Configuración compartida (`app/core/config.py`)

Toda la app usa **128×128** y parámetros unificados:

| Constante | Valor | Uso |
|-----------|--------|-----|
| `TARGET_SIZE` / `TARGET_SHAPE` | 128, (128,128) | Resolución de esqueletos (menor memoria). |
| `MIN_BRANCH_LENGTH` | 10 | Poda de espolones (plantillas y alumno). |
| `MAX_POINTS_TRAJECTORY` | 64 | Submuestreo para DTW (reducir N, M). |
| `PROCRUSTES_N_POINTS` | 50 | Puntos para alineación Procrustes. |
| `DTW_BAND_RATIO` | 0.25 | Ventana Sakoe-Chiba (no materializar matriz N×M). |
| `HAUSDORFF_TOLERANCE` / `HAUSDORFF_FACTOR` | 5, 2.0 | Score Hausdorff escalado a 128. |

---

## 5. Funcionamiento de las métricas

Las métricas se calculan en `app/metrics/` y se combinan en `app/metrics/scorer.py`.

### 5.1 Métricas geométricas (`geometric.py`)

Comparan **forma y posición** en **128×128**. Incluyen **SSIM** (en lugar de IoU), **Procrustes** y **Hausdorff**.

| Métrica | Descripción | Interpretación |
|---------|-------------|----------------|
| **ssim** / **ssim_score** | Structural Similarity Index entre ambos esqueletos (rango [-1,1] y score 0–100). | Más estable que IoU ante pequeñas variaciones. |
| **procrustes_disparity** / **procrustes_score** | Alineación óptima (escala, rotación, traslación) por Procrustes; disparity = M², score 0–100. | Forma global; más tolerante que Hausdorff (orientación a tutor). |
| **hausdorff** / **score** | Distancia de Hausdorff y score `max(0, 100 - (haus - tol) * factor)` con `tol=5`, `factor=2`. | Complementa a Procrustes (sensibilidad al peor punto). |

- Las secuencias se remuestrean a `PROCRUSTES_N_POINTS` (50) para Procrustes. Si los esqueletos tienen distinto tamaño, se redimensionan al patrón.

### 5.2 Topología (`topologic.py`)

Evalúa la **estructura discreta** del trazo: bucles, puntas y cruces.

| Campo        | Descripción |
|-------------|-------------|
| **loops**   | Número de contornos “hijos” (bucles cerrados) detectados en el esqueleto. |
| **endpoints** | Píxeles con exactamente 1 vecino (extremos de ramas). |
| **junctions** | Píxeles con 3 o más vecinos (cruces). |

- **match**: `true` si el número de **loops** del patrón y del alumno coinciden; si no, la estructura de bucles es distinta (ej. "a" con un ojal vs sin ojal).
- En el score final se usa como **todo o nada**: coincidencia de bucles = 100, si no = 30.

### 5.3 Trayectoria (`trajectory.py`)

Aproxima si el **orden/dirección del trazo** se parece al de la plantilla, con **menor uso de memoria**:

- **Secuencia**: puntos del esqueleto ordenados por ángulo respecto al centroide.
- **Submuestreo**: cada secuencia se limita a **MAX_POINTS_TRAJECTORY (64)** puntos para reducir N y M.
- **DTW con banda (Sakoe-Chiba)**: solo se rellena una banda \|i − j\| ≤ `band` (ratio 0.25), sin materializar la matriz N×M completa. Memoria O(n × band).
- **Salida**: `trajectory_error` (distancia DTW normalizada por longitud del camino). Menor = más similar; 999 si no hay puntos.

### 5.4 Similitud de coseno por segmentos (`segment_cosine.py`)

- Se divide cada esqueleto (ordenado por ángulo) en **12 segmentos** y se obtiene un **vector dirección** por segmento (inicio→fin, unitario).
- **Fórmula**: S(A,B) = cos(θ) = (A·B)/(‖A‖ ‖B‖). Se promedia sobre segmentos correspondientes.
- **Salida**: `cosine` (valor en [-1, 1]) y `score` (mapeado a 0–100). Compara ángulos de trazo, no solo coordenadas.

---

### 5.5 Calidad (`quality.py`)

Métricas **solo del trazo del alumno** (no comparan con la plantilla).

| Métrica            | Descripción |
|--------------------|-------------|
| **aspect_ratio**   | Ancho / alto del bounding box del esqueleto. Útil para ver si la letra está aplastada o estirada. |
| **shape_fingerprint** | Primeros 3 momentos de Hu (transformación logarítmica). Firma geométrica invariante a escala/rotación. |

No entran directamente en el score final; sirven para análisis o feedback adicional.

### 5.6 Score final (`scorer.py`)

Ponderación orientada a **tutor** (no castigar severamente); Procrustes tiene más peso que Hausdorff.

| Componente | Peso | Cálculo (resumido) |
|------------|------|---------------------|
| **Procrustes** | 28% | `procrustes_score` (0–100). |
| **Hausdorff** | 12% | `score_h = max(0, 100 - (hausdorff - HAUSDORFF_TOLERANCE) * HAUSDORFF_FACTOR)` (tol=5, factor=2). |
| **SSIM** | 20% | `ssim_score` (0–100). |
| **Topología** | 25% | 100 si `loops` coinciden, 30 si no. |
| **Trayectoria** | 10% | `score_traj = max(0, 100 - trajectory_error * 3)`. |
| **Coseno por segmentos** | 5% | `segment_cosine.score` (0–100). |

**Score final** = suma ponderada, redondeado a 2 decimales.

### 5.7 Imagen de comparación (`visualizer.py`)

El campo `image_b64` es una imagen PNG en Base64 que superpone ambos esqueletos:

- **Amarillo**: píxeles donde coinciden plantilla y alumno (acierto).
- **Verde oscuro**: solo plantilla (guía no cubierta).
- **Rojo**: solo alumno (trazo fuera de la plantilla / error).

Título: `Evaluación: {score_final}%`. Tamaño: **128×128** (`TARGET_SHAPE`).

## 6. Uso de memoria actual

Con **128×128** y **submuestreo + DTW con banda**, el uso de memoria por imagen es acotado y bajo.

### Por solicitud (una imagen)

| Dato | Tamaño aprox. | Notas |
|------|----------------|--------|
| Imagen decodificada | Variable (W×H bytes) | Solo durante preprocesamiento. |
| **Esqueletos** (patrón + alumno) | 2 × (128×128) = **32 KB** | uint8. Antes 256×256 eran 128 KB. |
| **Plantilla** (si no estaba en cache) | 16 KB | Un .npy 128×128. |
| **Procrustes** | 50×2 × 8 × 2 ≈ 1,6 KB | Dos arrays de 50 puntos (float64). |
| **Trayectoria (DTW)** | O(64 × band) | Band ≈ 16 → solo ~1 KB de celdas en banda (no matriz 64×64). |
| **Hausdorff** | Puntos ~500–2000 × 2 × 8 ≈ 8–32 KB | Por esqueleto; temporal. |
| **Overlay (visualizer)** | 128×128×3 = **48 KB** | RGB. |
| **SSIM / topología / calidad** | Del orden de 100–500 KB | Buffers 128×128 y filtros. |

**Total típico por request**: del orden de **5–15 MB** (incl. imagen de entrada y buffers intermedios), con pico muy por debajo de la versión 256×256 + matriz cdist completa.

### Comparación con la versión anterior

- **256×256**: dos esqueletos 128 KB, matriz cdist 64×64 o 1000×1000 (hasta ~8 MB).
- **128×128**: dos esqueletos 32 KB; DTW con banda evita la matriz N×M; submuestreo a 64 puntos reduce trabajo y memoria.

---

## 7. Cacheado de plantillas

- **Dónde**: en `app/api/endpoints.py`, el diccionario **`TEMPLATE_CACHE`** (clave = carácter, valor = array del esqueleto 128×128).
- **Cuándo se llena**: en la primera petición que pide un carácter, se llama a **`get_template(char)`**: si el carácter no está en cache, se hace `np.load(ruta_del_.npy)`, se redimensiona a 128×128 si el archivo era 256×256 y se guarda en `TEMPLATE_CACHE[char]`.
- **Peticiones siguientes**: para el mismo carácter se devuelve directamente la referencia en memoria; no hay lectura de disco ni nuevo redimensionado.
- **Uso de memoria del cache**: como máximo 62 caracteres × 16 KB ≈ **1 MB** si se cargan todas las plantillas a lo largo del tiempo. Solo se cargan las que se piden.
- **Ventajas**: menos I/O, menor latencia en requests repetidas y uso de memoria acotado por carácter.

## 8. Estructura de archivos relevante

```
app/
├── main.py                      # FastAPI app, monta el router
├── api/endpoints.py             # POST /evaluate, get_template (cache), orquesta métricas
├── core/
│   ├── config.py                # TARGET_SHAPE 128×128, MIN_BRANCH_LENGTH, DTW_BAND, etc.
│   └── processor.py             # preprocess_robust, prune_skeleton (pipeline alineado)
├── metrics/
│   ├── geometric.py             # SSIM, Procrustes, Hausdorff
│   ├── topologic.py             # bucles, endpoints, junctions
│   ├── trajectory.py            # submuestreo + DTW con banda (Sakoe-Chiba)
│   ├── segment_cosine.py        # similitud de coseno por segmentos
│   ├── quality.py               # aspect_ratio, Hu moments
│   └── scorer.py                # score final ponderado (Procrustes > Hausdorff)
├── utils/visualizer.py          # generate_comparison_plot → Base64 (128×128)
├── scripts/generate_templates.py  # plantillas 128×128, pipeline alineado con processor
├── templates/                   # .npy y .png por carácter (128×128)
└── fonts/                      # fuentes para generar plantillas
```